# 1.Import Data

In [3]:
import pandas as pd
import numpy as np
import json

In [4]:
data = np.abs(pd.read_csv('Data/Raw/data_all.csv'))
data= data.rename(columns = {'wspd':'prev_wspd','nxt_wspd':'wspd'})
data

,prev_displ,prev_vell,prev_accl,prev_wspd,displ,vell,accl,wspd,act,dispm,velm,accm,dispm_0,velm_0,accm_0
0,47,48,53,6,26,47,54,6,10.0,0.000107,0.000142,0.000336,0.000108,0.000142,0.000336
1,47,48,53,6,26,47,54,6,20.0,0.000107,0.000142,0.000336,0.000108,0.000142,0.000336
2,47,48,53,6,26,46,53,6,30.0,0.000107,0.000142,0.000336,0.000108,0.000142,0.000336
3,47,48,53,6,26,46,53,6,40.0,0.000107,0.000142,0.000336,0.000108,0.000142,0.000336
4,47,48,53,6,27,46,53,6,50.0,0.000107,0.000146,0.000336,0.000108,0.000142,0.000336
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23835,62,33,49,18,55,35,54,17,160.0,0.000126,0.000283,0.001810,0.000354,0.001104,0.003732
23836,62,33,49,18,55,36,54,17,170.0,0.000125,0.000265,0.001739,0.000354,0.001104,0.003732
23837,62,33,49,18,55,37,54,17,180.0,0.000124,0.000258,0.001710,0.000354,0.001104,0.003732
23838,62,33,49,18,61,35,54,17,190.0,0.000153,0.000352,0.002027,0.000354,0.001104,0.003732


In [5]:
(data['accm'] - data['accm_0']).mean(),(data['velm'] - data['velm_0']).mean(),(data['dispm'] - data['dispm_0']).mean()

(-3.591109614935337e-05, -0.0001538535311560804, -3.179069586526658e-05)

In [6]:
#reward function = Ucondi/Uuncondi if 
for j in ['disp','vel','acc']:
    data['r_'+j] = np.abs(data[j+'m']/data[j+'m_0'])
    # if no change reward = 0
    data.loc[data['r_'+j]==1,'r_'+j] = 0 - data['act']/2000
    data.loc[data['r_'+j]>1,'r_'+j] = -data.loc[data['r_'+j]>1,'r_'+j] - data['act']/2000
    data.loc[(data['r_'+j]<1)&(data['r_'+j]>0),'r_'+j] = 1/data.loc[data['r_'+j]<1,'r_'+j] - data['act']/2000
    
    

# 2. Create Subset data

## 2.1 Check Data summary for rewards

In [7]:
for j in ['disp','vel','acc']:
    print(data['r_'+j].mean())
    print(data['r_'+j].median())
    print()

0.8242301184103439
1.135145164224478

1.2643241780175236
1.4523596800356575

0.26639474560662185
0.9488653320026676



# 2.2 Separate the Data for models

In [8]:
for j in ['disp','vel','acc']:
    col_names = ['prev_'+j+'l', 'prev_wspd', j+'l', 'wspd', 'act', 'r_'+j]
    globals()[j] = round(data[col_names],6)
    eval(j)['state'] = list(zip( np.abs(eval(j)['prev_'+j+'l']),  eval(j)['prev_wspd']))
    eval(j)['state_p'] = list(zip( np.abs(eval(j)[j+'l']),  eval(j)['wspd']))
    globals()[j] = eval(j)[['state', 'state_p', 'act', 'r_'+j]].drop_duplicates()
    eval(j).to_csv('Data/{}.csv'.format(j))
# we have three datasets: disp, vel, acc, with s, s', a r
disp

,state,state_p,act,r_disp
0,"(47, 6)","(26, 6)",10.0,1.005257
1,"(47, 6)","(26, 6)",20.0,1.000257
2,"(47, 6)","(26, 6)",30.0,0.995256
3,"(47, 6)","(26, 6)",40.0,0.990256
4,"(47, 6)","(27, 6)",50.0,0.985256
...,...,...,...,...
23835,"(62, 18)","(55, 17)",160.0,2.731185
23836,"(62, 18)","(55, 17)",170.0,2.746416
23837,"(62, 18)","(55, 17)",180.0,2.760593
23838,"(62, 18)","(61, 17)",190.0,2.219886


In [9]:
#check the total states vs observed next states
for j in [disp,vel,acc]:
    print('total samples:',len(j))
    print('total unique current state:', len(set(j['state'])))
    print('total unique next state:',len(set(j['state_p'])))
    print()

total samples: 22930
total unique current state: 641
total unique next state: 1268

total samples: 23099
total unique current state: 424
total unique next state: 868

total samples: 21967
total unique current state: 338
total unique next state: 814



In [10]:
for i in data.columns:
    print(i,':',len(set(data[i])))
# 20 actions
# 395 displacement
# 675 velocity
# 1026 acceleration

prev_displ : 85
prev_vell : 67
prev_accl : 59
prev_wspd : 20
displ : 98
vell : 82
accl : 87
wspd : 20
act : 20
dispm : 22301
velm : 22781
accm : 9863
dispm_0 : 1188
velm_0 : 1191
accm_0 : 1190
r_disp : 23338
r_vel : 23739
r_acc : 19696


## 2.3 Unique Value of (parameter, wind_speed)

In [11]:
for i in ['displ','vell','accl']:
    temp = set(pd.concat([data['prev_'+i],data[i]], axis=0,ignore_index=True))
    print(len(temp))
    temp2 = set(pd.concat([data['prev_wspd'],data['wspd']], axis=0,ignore_index=True))
    globals()['unique_{}'.format(i)] = [(j,k) for j in range(1,101) for k in range(1,21)]
len(unique_displ),len(unique_vell),len(unique_accl)
#(2000, 2000, 2000)

98
82
87


(2000, 2000, 2000)

# 4. Dictionarized all (disp/vel/acc, wind) to s1,s2...

In [12]:
#create dictionary for each state
for i in ['unique_displ','unique_vell','unique_accl']:
  my_list = eval(i)
  globals()['dict_'+i] = dict() 
  for index,value in enumerate(my_list):
    eval('dict_'+i)[index+1] = value
#dict_unique_prev_displ
len(dict_unique_displ.keys())
dict_unique_displ

{1: (1, 1),
 2: (1, 2),
 3: (1, 3),
 4: (1, 4),
 5: (1, 5),
 6: (1, 6),
 7: (1, 7),
 8: (1, 8),
 9: (1, 9),
 10: (1, 10),
 11: (1, 11),
 12: (1, 12),
 13: (1, 13),
 14: (1, 14),
 15: (1, 15),
 16: (1, 16),
 17: (1, 17),
 18: (1, 18),
 19: (1, 19),
 20: (1, 20),
 21: (2, 1),
 22: (2, 2),
 23: (2, 3),
 24: (2, 4),
 25: (2, 5),
 26: (2, 6),
 27: (2, 7),
 28: (2, 8),
 29: (2, 9),
 30: (2, 10),
 31: (2, 11),
 32: (2, 12),
 33: (2, 13),
 34: (2, 14),
 35: (2, 15),
 36: (2, 16),
 37: (2, 17),
 38: (2, 18),
 39: (2, 19),
 40: (2, 20),
 41: (3, 1),
 42: (3, 2),
 43: (3, 3),
 44: (3, 4),
 45: (3, 5),
 46: (3, 6),
 47: (3, 7),
 48: (3, 8),
 49: (3, 9),
 50: (3, 10),
 51: (3, 11),
 52: (3, 12),
 53: (3, 13),
 54: (3, 14),
 55: (3, 15),
 56: (3, 16),
 57: (3, 17),
 58: (3, 18),
 59: (3, 19),
 60: (3, 20),
 61: (4, 1),
 62: (4, 2),
 63: (4, 3),
 64: (4, 4),
 65: (4, 5),
 66: (4, 6),
 67: (4, 7),
 68: (4, 8),
 69: (4, 9),
 70: (4, 10),
 71: (4, 11),
 72: (4, 12),
 73: (4, 13),
 74: (4, 14),
 75: (4, 

## 4.1 Conver the tuple states into a dictionarized states 

In [13]:
for i in ['disp','vel','acc']:
    print(i)
    index = 0
    temp = eval(i)
    dic_i = eval('dict_unique_'+i+'l')
    key_list = list(dic_i.keys())
    val_list = list(dic_i.values())

    temp['s'] = 0
    temp['s_p'] = 0

    for z in range(len(temp)):
        j = temp['state'].iloc[z]
        k = temp['state_p'].iloc[z]
        position_s = val_list.index(j)
        position_s_p = val_list.index(k)
        temp['s'].iloc[index] = key_list[position_s]
        temp['s_p'].iloc[index] = key_list[position_s_p]
        index +=1
    temp['a'] = temp['act']/10
    temp['a']  = temp['a'].astype(int)
    globals()[i] = temp
#my_dict_unique_displ.get()

disp


/opt/homebrew/Caskroom/miniforge/base/envs/tensorflow/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


vel
acc


In [14]:
disp

,state,state_p,act,r_disp,s,s_p,a
0,"(47, 6)","(26, 6)",10.0,1.005257,926,506,1
1,"(47, 6)","(26, 6)",20.0,1.000257,926,506,2
2,"(47, 6)","(26, 6)",30.0,0.995256,926,506,3
3,"(47, 6)","(26, 6)",40.0,0.990256,926,506,4
4,"(47, 6)","(27, 6)",50.0,0.985256,926,526,5
...,...,...,...,...,...,...,...
23835,"(62, 18)","(55, 17)",160.0,2.731185,1238,1097,16
23836,"(62, 18)","(55, 17)",170.0,2.746416,1238,1097,17
23837,"(62, 18)","(55, 17)",180.0,2.760593,1238,1097,18
23838,"(62, 18)","(61, 17)",190.0,2.219886,1238,1217,19


In [15]:
for i in ['disp','vel','acc']:
    globals()[i+'o'] = eval(i)[['s','a','r_'+i,'s_p']].rename(columns={ 'r_'+i: 'r','s_p' : 'sp'})

In [16]:
print(len(set(acco['s'])))

338


In [17]:
for i in ['disp','vel','acc']:
    idx = eval(i+'o').groupby(['s'])['r'].transform(max) == eval(i+'o')['r']
    globals()[i+'_optimal'] = eval(i+'o')[idx]
    print(len(set(eval(i+'o')['s'])))
    globals()[i+'_act_count'] = eval(i+'_optimal')['a'].value_counts()

641
424
338


In [18]:
pd.concat([disp_act_count,vel_act_count,acc_act_count],axis =1)

,a,a,a
1,77,13,187
2,31,17,33
3,40,20,18
4,52,27,13
5,82,43,21
6,53,39,16
7,59,30,10
8,36,36,8
9,27,35,3
10,13,13,1


# 4.2 Save the Data to Data file
* csv
* Dictionary

In [21]:
for i in ['disp','vel','acc']:
    eval(i+'o').to_csv('Data/Version_from_python/{}.csv'.format(i+'o'),index=False)

    dic_i = eval('dict_unique_'+i+'l')
    a_file = open("Data/Version_from_python/{}_dic.json".format(i), "w")
    json.dump(dic_i, a_file)
    a_file.close()

### Loading Test

In [22]:
for i in ['disp','vel','acc']:
    a_file = open("Data/Version_from_python/{}_dic.json".format(i), "r")
    output = a_file.read()
    globals()['{}_dic'.format(i)]=output
#dict_unique_displ

In [23]:
a_file = open("Data/Version_from_python/disp_dic.json", "r")
a_dictionary = json.load(a_file)

In [24]:
a_dictionary['1']

[1, 1]

In [ ]:
len(disp_dic)
disp_dic[1]

'"'

In [ ]:
# ratio =  max(DO) / max(NOTHING)
    # if ratio = 1 -> reward = 0 + penalty
    # if ratio > 1 -> reward = -ratio + penalty
    # if ratio < 1 -> reward = 1/ratio + penalty

# reward = max(NOTHING) / max(DO) - Voltage / constant 